NHS Digital have published their [methodology for high risk shielded patient list identification](https://digital.nhs.uk/coronavirus/shielded-patient-list/methodology/medicines-data). They provided the list as BNF codes and following notebook generates codes compliant with the NHS Dictionary of Medicines and Devices which is the local UK Snomed drug extension. We will use this rule to identify severe asthma

In [1]:
from ebmdatalab import bq
import os
import pandas as pd

### Inclusion Criteria
NHS Digital inclusion criteria rule 1 states "Patients with asthma were identified as being prescribed Long acting beta2-agonist (LABA) as either a LABA or in combination with an inhaled corticosteroid (LABA/ICS) OR prescriptions for a leukotriene receptor antagonist (e.g. monteluekast).""

In [2]:
## The following is written based on version 1 frm March 27th and 
## is archived at https://web.archive.org/save/https://digital.nhs.uk/coronavirus/shielded-patient-list/methodology/medicines-data

sql = '''
WITH bnf_codes AS (  
  SELECT bnf_code FROM hscic.presentation WHERE 
  (bnf_code LIKE '030302%' OR #BNF leukotriene antagonists
  bnf_code LIKE '0301011B0%' OR #BNF bambuterol
  bnf_code LIKE '0301011E0%' OR #BNF formoterol
  bnf_code LIKE '0301011U0%' OR #BNF salmeterol
  bnf_code LIKE '0302000C0%' OR #BNF Beclometasone dipropionate
  bnf_code LIKE '0302000K0%' OR #BNF budesonide
  bnf_code LIKE '0302000V0%' OR #BNF Fluticasone furoate 
  bnf_code LIKE '0302000N0%') #BNF Fluticasone propionate 
   )
   
SELECT "vmp" AS type, id, bnf_code, nm
FROM dmd.vmp
WHERE bnf_code IN (SELECT * FROM bnf_codes)'''

severe_asthma_inc_rule1 = bq.cached_read(sql, csv_path=os.path.join('..','data','severe_asthma_inc_rule1.csv'))
pd.set_option('display.max_rows', None)
severe_asthma_inc_rule1.head(10)

,type,id,bnf_code,nm
0,vmp,35938011000001101,0301011U0AAABAB,Salmeterol 50microgram inhalation powder blisters
1,vmp,35937811000001108,0301011U0AAAAAA,Salmeterol 50microgram inhalation powder blist...
2,vmp,35907611000001101,0302000C0AAALAL,Beclometasone 100microgram inhalation powder b...
3,vmp,35907711000001105,0302000C0AAAGAG,Beclometasone 100microgram inhalation powder b...
4,vmp,35908311000001107,0302000C0AAAHAH,Beclometasone 200microgram inhalation powder b...
5,vmp,35908211000001104,0302000C0AAAKAK,Beclometasone 200microgram inhalation powder b...
6,vmp,35909011000001104,0302000C0AAAJAJ,Beclometasone 400microgram inhalation powder b...
7,vmp,35908911000001108,0302000C0AAAQAQ,Beclometasone 400microgram inhalation powder b...
8,vmp,12911011000001100,0302000C0AABXBX,Beclometasone 100micrograms/dose / Formoterol ...
9,vmp,14959511000001107,0302000K0AAAZAZ,Budesonide 100micrograms/dose inhaler CFC free


NHS Digital inclusion criteria rule 2 states "From the above list of patients, those who had been dispensed 4 or more prescriptions for prednisolone between July 2019 and December 2019 were identified and considered to have severe asthma. "

We may wish to consider a different criteria for dispensed. e.g. we could do must have had at least 60 x 5mg prenisolone tabs in last three months

In [3]:
sql = '''
WITH bnf_codes AS (  
  SELECT bnf_code FROM hscic.presentation WHERE 
  bnf_code LIKE '0603020T0%' #BNF oral prednisolone
   )
SELECT "vmp" AS type, id, bnf_code, nm
FROM dmd.vmp
WHERE bnf_code IN (SELECT * FROM bnf_codes)'''

severe_asthma_inc_rule2 = bq.cached_read(sql, csv_path=os.path.join('..','data','severe_asthma_inc_rule2.csv'))
pd.set_option('display.max_rows', None)
severe_asthma_inc_rule2.head(10)


,type,id,bnf_code,nm
0,vmp,8672011000001109,0603020T0AABEBE,Prednisolone 2mg/5ml oral suspension
1,vmp,13078411000001108,0603020T0AABBBB,Prednisolone 1.5mg/5ml oral suspension
2,vmp,13078311000001101,0603020T0AABBBB,Prednisolone 1.5mg/5ml oral solution
3,vmp,13078611000001106,0603020T0AABCBC,Prednisolone 10mg/5ml oral suspension
4,vmp,429995001,0603020T0AABCBC,Prednisolone 10mg/5ml oral solution
5,vmp,432225008,0603020T0AAAYAY,Prednisolone 15mg/5ml oral suspension
6,vmp,416533002,0603020T0AAAYAY,Prednisolone 15mg/5ml oral solution
7,vmp,13079011000001109,0603020T0AABFBF,Prednisolone 1mg/5ml oral suspension
8,vmp,13078911000001100,0603020T0AABFBF,Prednisolone 1mg/5ml oral solution
9,vmp,13079211000001104,0603020T0AABIBI,Prednisolone 2.5mg/5ml oral suspension


**Severe Asthma**  = <br/> 
Presence of at least one from `severe_asthma_inc_rule1` in last three months <br/>
AND <br/>
X issues of at least one from `severe_asthma_inc_rule2` in the last three months